<a href="https://colab.research.google.com/github/rajibmondal/Bengali.AI-Handwritten-Grapheme-Classification/blob/master/Bengali_AI_Handwritten_Grapheme_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rajibmondal","key":"5bdef41fd479b5e91509300e5efde146"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# !kaggle competitions download -c digit-recognizer
!kaggle competitions download -c bengaliai-cv19

  0% 0.00/4.72k [00:00<?, ?B/s]
100% 4.72k/4.72k [00:00<00:00, 7.43MB/s]
  0% 0.00/1.18M [00:00<?, ?B/s]
100% 1.18M/1.18M [00:00<00:00, 69.4MB/s]
  0% 0.00/1.30M [00:00<?, ?B/s]
100% 1.30M/1.30M [00:00<00:00, 187MB/s]
 99% 980M/986M [00:12<00:00, 60.3MB/s]
100% 986M/986M [00:12<00:00, 79.7MB/s]
  0% 0.00/1.70k [00:00<?, ?B/s]
100% 1.70k/1.70k [00:00<00:00, 1.51MB/s]
  0% 0.00/1.32M [00:00<?, ?B/s]
100% 1.32M/1.32M [00:00<00:00, 191MB/s]
 98% 969M/992M [00:14<00:00, 55.9MB/s]
100% 992M/992M [00:14<00:00, 70.4MB/s]
100% 991M/993M [00:17<00:00, 35.3MB/s]
100% 993M/993M [00:17<00:00, 60.8MB/s]
  0% 0.00/16.8k [00:00<?, ?B/s]
100% 16.8k/16.8k [00:00<00:00, 45.2MB/s]
100% 991M/991M [00:25<00:00, 9.25MB/s]
100% 991M/991M [00:25<00:00, 40.6MB/s]
  0% 0.00/4.75k [00:00<?, ?B/s]
100% 4.75k/4.75k [00:00<00:00, 4.85MB/s]
  0% 0.00/944 [00:00<?, ?B/s]
100% 944/944 [00:00<00:00, 3.20MB/s]
  0% 0.00/1.28M [00:00<?, ?B/s]
100% 1.28M/1.28M [00:00<00:00, 83.4MB/s]
  0% 0.00/1.25M [00:00<?, ?B/s]
100% 1.

In [5]:
ls

class_map_corrected.csv        test_image_data_2.parquet.zip
class_map.csv                  test_image_data_3.parquet.zip
kaggle.json                    train.csv.zip
sample_data/                   train_image_data_0.parquet.zip
sample_submission.csv          train_image_data_1.parquet.zip
test.csv                       train_image_data_2.parquet.zip
test_image_data_0.parquet.zip  train_image_data_3.parquet.zip
test_image_data_1.parquet.zip  train_multi_diacritics.csv


In [6]:
# from google.colab import drive
# drive.mount('/content/gdrive')

!unzip "/content/train.csv.zip"

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2

from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns

Using TensorFlow backend.


In [0]:
train_df_ = pd.read_csv('/content/train.csv')
test_df_ = pd.read_csv('/content/test.csv')
class_map_df = pd.read_csv('/content/class_map.csv')
sample_sub_df = pd.read_csv('/content/sample_submission.csv')

In [9]:
print(f'Size of training data: {train_df_.shape}')
print(f'Size of test data: {test_df_.shape}')
print(f'Size of class map: {class_map_df.shape}')

Size of training data: (200840, 5)
Size of test data: (36, 3)
Size of class map: (186, 3)


In [0]:
#Explore Data Analysis

HEIGHT=240
WIDTH=280

def get_n(df, field, n, top=True):
  top_graphemes = df.groupby([field]).reset_index(name='counts')['counts'].sort_values(ascending=not top)[:n]
  top_grapheme_roots = top_graphemes.index
  top_grapheme_counts = top_graphemes.values
  top_graphemes = class_map_df[class_map_df['component_type'] == field].reset_index().iloc[top_grapheme_roots]
  top_graphemes.drop(['component_type','label'], axis=1, inplace=True)
  top_graphemes.loc[:, 'count'] = top_grapheme_counts
  return top_graphemes